In [1]:
import requests
import os
from os import path
import zipfile
from datetime import datetime
from shutil import copyfile

In [2]:
class Retrosheet():
    
    def __init__(self):
        self.now = datetime.now()
        self.log = 'C:\\retro\\ProcessLog\\log_{}{}{}.txt'.format(self.now.year, self.now.month, self.now.day)
        self.directories = []

    def get_file(self,startyear, endyear, file_type, inc):
        
        self.directories = []
                      
        if path.exists(self.log):
            logfile = open(self.log, 'a')
            logfile.write('\n\n Processing Bevent as of: {}{}{}'.format(self.now.year, self.now.month, self.now.day))
        else:
            logfile = open(self.log, 'w')
            logfile.write('New file started:\n\nProcessing Bevent as of: {}{}{}'.format(self.now.year, self.now.month, self.now.day))
        
        numerrors = 0
        goodFile = True
        
        if file_type == 'reg':
            tag = 'eve.zip'
        elif file_type == 'post':
            tag = 'post.zip'
        elif file_type == 'decade':
            tag =  'seve.zip'
        else:
            goodFile = False
        
        if goodFile:
        
            for y in range(startyear, endyear + 1, inc):
                error = 0
                
                src = 'C:\\retro\\RawFiles'
                
                if not path.exists(src[:8]):
                    os.mkdir(src[:8])
                
                if not path.exists(src):
                    os.mkdir(src)
                
                url = "https://www.retrosheet.org/events/{}{}".format(y, tag)
                directory_to_extract_to = 'C:\\retro\\RawFiles\\{}_{}'.format(y, file_type)
                
                if file_type == 'reg':
                    filepath = 'C:\\retro\\RawFiles\\{}'.format(url[-11:])
                else:
                    filepath = 'C:\\retro\\RawFiles\\{}'.format(url[-12:])
                    
                #If the download doesn't already exist, go get it
                if not path.exists(filepath):
                    
                    r = requests.get(url)
                    
                    try:
                        r.raise_for_status()
                    except Exception as exc:
                        error = 1
                        numerrors += 1
                        logfile.write('\n\nThere was a problem: {}'.format(exc))
                    
                    if error == 0:
                        with open(filepath, 'wb') as f:
                            f.write(r.content)
                        
                        logfile.write('\n\nSuccessfully downloaded {}'.format(filepath))
                        
                        if not path.exists(directory_to_extract_to):
                            
                            os.mkdir(directory_to_extract_to)
                            
                            with zipfile.ZipFile(filepath, 'r') as zip_ref:
                                zip_ref.extractall(directory_to_extract_to)
                                
                            copyfile('C:\\Users\\Taylor.Douglas\\Documents\\retro\\RawFiles\\BEVENT.exe', 
                                            path.join(directory_to_extract_to, 'BEVENT.exe'))
                        
                        else:
                            logfile.write('\n\n{} not unzipped'.format(filepath))
                        
                else:
                        logfile.write('\n\n{} is already downloaded'.format(filepath))
                
                self.directories.append(directory_to_extract_to)
                
                if numerrors >= 3:
                    logfile.write('\n\nError limit of 3 exceeded. Process terminated')
                    break
        
        logfile.close()
        
    def run_bevent_tocsv(self, fieldlist = [[0,96]]):
        
        directories = self.directories
        
        if path.exists(self.log):
            logfile = open(self.log, 'a')
            logfile.write('\n\n Processing Bevent as of: {}{}{}'.format(self.now.year, self.now.month, self.now.day))
        else:
            logfile = open(self.log, 'w')
            logfile.write('New file started:\n\nProcessing Bevent as of: {}{}{}'.format(self.now.year, self.now.month, self.now.day))
        
        
        fields = ''
    
        i = 0
        
        if type(fieldlist) == list:
            for field in fieldlist:
                if i == 0:
                    if type(field) == list and len(field) > 1:
                        fields = '-f ' + fields + str(field[0]) + '-' + str(field[1])
                    elif type(field) == list and len(field) == 1:
                        fields = '-f ' + fields + str(field[0])
                    else:
                        fields = '-f ' + fields + str(field)
                if i >= 1:
                    if type(field) == list and len(field) > 1:
                        fields = '{}, {}-{}'.format(fields, field[0], field[1])
                    elif type(field) == list and len(field) == 1:
                        fields = '{}, {}'.format(fields, field)
                    else:
                        fields = '{}, {}'.format(fields, field)
                
                i += 1
        
        elif type(fieldlist) == int:
            fields = '-f ' + str(fieldlist)        
                    
        for d in directories:
        
            os.chdir(d)
            
            landing = 'C:\\retro\\RawFiles\\csvs'
            
            if not path.exists(landing):
                os.mkdir(landing)
        
            for (dirname, dirs, files) in os.walk(d):
                for filename in files:
                    if filename[-3:] == 'EVA' or filename[-3:] == 'EVN' or filename[-3:] == 'EVE':
                    #if filename[0:4] == '2010' and filename[-3:] == 'EVA':
                        
                        if d[-4:] == 'post' and d[4:6] == 'WS':
                            csvfile = filename[0:6] + '_post.csv'
                        elif d[-4:] == 'post':
                            csvfile = filename[0:7] + '_post.csv'
                        else:
                            csvfile = filename[0:7] + '.csv'
                        
                        year = filename[0:4]
                        if not path.exists(path.join(landing, csvfile)):
                            cmd = 'bevent -y {} {} {} >{}'.format(year, fields, filename, path.join(landing, csvfile))
                            os.system(cmd)
                            logfile.write('\n\nExecuted {} for {}.'.format(cmd, filename))
                            #print('Executed ' + cmd + ' for ' + filename)
                        #cmd = 'bevent -y ' + year + ' -f 0-96 ' + filename + ' >FileOutput\\' + thefile + '.csv'
                        #print(thefile)
                        else:
                            logfile.write('\n\n{} already exists.'.format(csvfile))
        
        logfile.close()
        
    def get_single(self, startyear, endyear = None, season_part = 'reg'):
        
        """ name: get_single
            desc: Iterates through a range of years and pulls associated Single Year Files from Retrosheet
            params: startyear -- an integer year
                    endyear -- an integer year > startyear. If endyear < startyear, it is automatically set
                                to the startyear value
                    season_part -- 'reg' for regular season
                                   'post' for postseason
                                   
                    *Note: Postseason files are only for single seasons
        """
        try:
                   
            if not endyear:
                endyear = startyear
            
            if endyear < startyear:
                endyear = startyear
            
            startyear = int(startyear)
            endyear = int(endyear)
            
            inc = 1
            
            self.get_file(startyear, endyear, season_part, inc)
            
        except ValueError as val:
            print('startyear and endyear should be integer values') 
            
    def get_decade(self, startyear, endyear = None):
        
        """ name: get_decade
            desc: Iterates through a range of decades and pulls associated Decade Files from Retrosheet
            params: startyear -- an integer year
                    endyear -- an integer year > startyear. If endyear < startyear, it is automatically set
                                to the startyear value
                                
                    *Note: Decade files only contain regular season data
        """
        
        try:
            
            startyear = int(startyear)
            endyear = int(endyear)
            
            if endyear < startyear:
                endyear = startyear
            
            startyear = startyear - int(str(startyear)[-1])
            
            if endyear:
                endyear = endyear - int(str(endyear)[-1])
            else:
                endyear = startyear
            
            file_type = 'decade'
            inc = 10
            
            self.get_file(startyear, endyear, file_type, inc)
                    
        except ValueError as val:
            print('startyear and endyear should be integer values')

    def single_tocsv(self, startyear, endyear = None, season_part = 'reg', fieldlist = [[0,96]]):
        
        self.get_single(startyear, endyear, season_part)
        
        self.run_bevent_tocsv(fieldlist)
        
    def decade_tocsv(self, startyear, endyear = None, fieldlist = [[0,96]]):
        
        self.get_decade(startyear, endyear)
        
        self.run_bevent_tocsv(fieldlist)

In [3]:
rc = Retrosheet()

In [4]:
rc.single_tocsv(1998, 2018)

In [5]:
rc.single_tocsv(1998, 2018, 'post', [[0, 96]])

In [ ]:
#rc.decade_tocsv(2000, 2009, [[0,10]])

In [ ]:
#rc.directories